# Set-up

## Installing libraries and libcudnn8

In [1]:
## only with GPU
!apt-get install --allow-change-held-packages libcudnn8=8.1.1.33-1+cuda11.2 -y
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases
!pip install scikeras[tensorflow]




Package libcudnn8 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Version '8.1.1.33-1+cuda11.2' for 'libcudnn8' was not found
  Cloning https://github.com/UN-GCPDS/python-gcpds.databases to /tmp/pip-req-build-jo9ju_6d
  Running command git clone --filter=blob:none --quiet https://github.com/UN-GCPDS/python-gcpds.databases /tmp/pip-req-build-jo9ju_6d
  Resolved https://github.com/UN-GCPDS/python-gcpds.databases to commit d174df9958b6638156dcfe03996a6307e631a6a2
  Preparing metadata (setup.py) ... done
  Created wheel for gcpds-databases: filename=gcpds_databases-0.2-py3-none-any.whl size=32972807 sha256=036aaab7c765f40fce7d745dbc122846133a373c0c88be0141cc5abda8ad0307
  Stored in directory: /tmp/pip-ephem-wheel-cache-p8o_beih/wheels/ae/48/8d/edf617d5fe8f03b17aa26306a04abdfcc605b218d8e6deac83
Successfully built gcpds-databases
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.

--2025-04-28 12:47:36--  https://docs.google.com/uc?export=download&confirm=&id=1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7
Resolving docs.google.com (docs.google.com)... 142.251.183.138, 142.251.183.113, 142.251.183.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.183.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7&export=download [following]
--2025-04-28 12:47:36--  https://drive.usercontent.google.com/download?id=1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.184.132, 2607:f8b0:4001:c66::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.184.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86900 (85K) [application/octet-stream]
Saving to: ‘MI_EEG_ClassMeth.zip’

MI_EEG_ClassMeth.zi 100%[===================>]  84.

## Import libraries

In [3]:
# gcpds datasets
from gcpds.databases import GIGA_MI_ME
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import gcpds.databases as loaddb

# freq filter 
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr

# general
import numpy as np
from scipy.signal import resample
import pickle
import warnings
import mne
from time import time
warnings.filterwarnings('ignore')

In [4]:
!git clone https://github.com/UN-GCPDS/eeg_models.git

import sys
sys.path.append('/kaggle/working/eeg_models')

from utils import train
from models.IRKCSFCnet import GFC_reg_renyi

Cloning into 'eeg_models'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 173 (delta 110), reused 88 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 105.86 KiB | 2.71 MiB/s, done.
Resolving deltas: 100% (110/110), done.


2025-04-28 12:47:48.271270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745844468.568042      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745844468.650678      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Define the model (Gaussian functional conectivity network)

In [5]:
import tensorflow as tf

In [6]:
# Parámetros específicos del modelo EEGNet
model_params = {"nb_classes":2, 
        "filters":2,
        "Chans" :64, 
        "Samples" :320}

# Argumentos generales del modelo
model_args = {
    'model_name': 'IRKCS-FCnet',
    'nb_classes': 2,
    'autoencoder': False
}

In [7]:
model= GFC_reg_renyi(**model_params)
model.summary()
# tf.keras.utils.plot_model(model,   show_shapes=True)

I0000 00:00:1745844485.059130      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745844485.059819      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)                   │ (None, 64, 320, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 64, 301, 2)          │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64, 301, 2)          │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 64, 301, 2)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gfc (GFC)                            │ (None, 2, 64, 64)           │               1 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_triu (get_triu)                  │ (None, 2, 2016, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 1, 2016, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1, 2016, 1)          │               4 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1, 2016, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2016)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2016)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ logits (Dense)                       │ (None, 2)                   │           4,034 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Activation)                  │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,087 (15.96 KB)

 Trainable params: 4,081 (15.94 KB)

 Non-trainable params: 6 (24.00 B)

# Experiment

## Experiment configuration 

In [8]:
# Nombre de la base de datos y carga
db_name = 'GIGA_MI_ME'
db_path = '/kaggle/input/giga-science-gcpds/GIGA_MI_ME'
db = GIGA_MI_ME(db_path)

# Frecuencia de muestreo
fs = db.metadata['sampling_rate']

# Nombres de los canales EEG
eeg_ch_names = [
    'Fp1', 'Fpz', 'Fp2',
    'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz'
]

# Definición de los bancos de frecuencia y ventana temporal
f_bank = np.array([[4., 40.]])        # Banda de frecuencia de interés
vwt = np.array([[2.5, 5]])            # Ventana temporal para la representación TF

# Representación tiempo-frecuencia
tf_repr = TimeFrequencyRpr(sfreq=fs, f_bank=f_bank, vwt=vwt)

# Argumentos para la función de carga de datos
load_args = {
    'db': db,
    'eeg_ch_names': eeg_ch_names,
    'fs': fs,
    'f_bank': f_bank,
    'vwt': vwt,
    'new_fs': 128.0,
    'tf_repr': tf_repr
}

# Selección de sujetos (excluyendo sujetos 29 y 34)
subjects = np.arange(1, db.metadata['subjects'] + 1)
subjects = np.delete(subjects, [28, 33])

In [9]:
# === Imports ===
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from sklearn.model_selection import StratifiedShuffleSplit

# === Parámetros generales ===
seed = 23
verbose = 0

# === Callbacks de entrenamiento ===
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=30,
    verbose=verbose,
    mode='min',
    min_delta=0.01,
    min_lr=0
)

terminate_on_nan = TerminateOnNaN()

callbacks = [reduce_lr_on_plateau, terminate_on_nan]

# === Configuración de validación cruzada ===
cv_args = {
    'cv': StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=seed)
}

# === Argumentos de compilación del modelo ===
compile_args = {
    'loss': SparseCategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'init_lr': 1e-2
}

# === Argumentos del ajuste (fit) del modelo ===
fit_args = {
    'epochs': 500,
    'verbose': verbose,
    # 'callbacks': callbacks
}

In [10]:
for sbj in subjects[:]:
  print('sbj = ', sbj)
  load_args['sbj'] = sbj
  results = train(model, db_name, tf_repr, load_args, cv_args, model_args, compile_args, fit_args, seed)

sbj =  1
Resampling from 512.000000 to 128.000000 Hz.


I0000 00:00:1745844496.687883     130 service.cc:148] XLA service 0x79438c10d040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745844496.688915     130 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745844496.688934     130 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745844497.140420     130 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745844501.142888     130 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 759ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 766ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 774ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 767ms/step
sbj =  2
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 792ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 762ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 774ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 766ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 792ms/step
sbj =  3
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 780ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 765ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 780ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 773ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 765ms/step
sbj =  4
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 862ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 768ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 779ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 778ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 767ms/step
sbj =  5
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━

In [11]:
results

{'params': [],
 'mean_acc': 0.66,
 'mean_kappa': 0.32,
 'mean_auc': 0.746,
 'mean_f1_left': 0.679,
 'mean_f1_right': 0.605,
 'mean_recall_left': 0.75,
 'mean_recall_right': 0.57,
 'mean_precision_left': 0.655,
 'mean_precision_right': 0.713,
 'std_acc': 0.082,
 'std_kappa': 0.163,
 'std_auc': 0.089,
 'std_f1_left': 0.0,
 'std_f1_right': 0.0,
 'std_recall_left': 0.0,
 'std_recall_right': 0.0,
 'std_precision_left': 0.0,
 'std_precision_right': 0.0}

In [12]:
import pickle

with open(f'results_{model_args["model_name"]}.pkl', 'wb') as f:
    pickle.dump(results, f)